In [1]:
import numpy as np 
import pandas as pd
import torch 
from bokeh.models import ColumnDataSource

import sys
import os
import pickle 
current_path = notebook_dir = os.getcwd()
working_dir = os.path.abspath(os.path.join(current_path, '..','..'))
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)

from constants.paths import FOLDER_PATH

## Get NetMob consumption around station 

In [2]:
from load_inputs.netmob_POIs_per_station import extract_apps_tags_modes
stations = ['AMP', 'BLA', 'BRO', 'COR', 'CRO', 'CUS', 'FLA', 'GAR', 'GIL', 'GRA', 'HEN', 'JAU', 'LAE', 'MAC', 'MER', 'PAR', 'PRY', 'SAN', 'SOI', 'VEN',
            'BEL', 'BON', 'CHA', 'CPA', 'CUI', 'DEB', 'FOC', 'GER', 'GOR', 'GUI', 'HOT', 'JEA', 'LUM', 'MAS', 'OGA', 'PER', 'REP', 'SAX', 'VAI', 'VMY']
Apps = ['Instagram', 'Deezer','Web_Weather','Web_Downloads','Google_Maps','Twitter','Tor','Microsoft_Office']


NetMob_selected_tags = ['station_epsilon300'] # 'station_epislon300'
data_dict_eps300 = {station_i:{app_j : {} for app_j in Apps} for station_i in stations}
for station_i in stations:
    for app_j in Apps:
        path_epsilon_around_station = f'POIs/netmob_POI_Lyon/InputsEpsilon/{station_i}'
        data_path,metadata_path = f'{FOLDER_PATH}/{path_epsilon_around_station}/data.npy', f'{FOLDER_PATH}/{path_epsilon_around_station}/metadata.pkl'  #contains 68 apps
        data_app = np.load(open(data_path,"rb"))
        metadata = pickle.load(open(metadata_path,"rb"))

        NetMob_selected_apps = [app_j]
        NetMob_transfer_mode = ['DL']
        data = extract_apps_tags_modes(data_app,metadata,NetMob_selected_apps,NetMob_selected_tags,NetMob_transfer_mode)
        data_dict_eps300[station_i][app_j] = data.reshape(-1)
if False:
    pickle.dump(data_dict,open('NetMob_samples_epsilon300.pkl','wb'))

## Down META du 11 Mai

In [ ]:
START = '03/16/2019'
END = '06/01/2019'
FREQ = '15min'
coverage_local = pd.date_range(start=START, end=END, freq=FREQ)[:-1]

pd.DataFrame(np.log(data_dict_eps300[station_i]['Instagram']),index = coverage_local).plot(title = 'Consumption Instagram in log scale')

In [ ]:
#data_dict_eps300 = pickle.load(open('NetMob_samples_epsilon300.pkl','rb'))
#data_dict_eps100 = pickle.load(open('NetMob_samples_epsilon100.pkl','rb'))

stations = ['AMP', 'BLA', 'BRO', 'COR', 'CRO', 'CUS', 'FLA', 'GAR', 'GIL', 'GRA', 'HEN', 'JAU', 'LAE', 'MAC', 'MER', 'PAR', 'PRY', 'SAN', 'SOI', 'VEN',
            'BEL', 'BON', 'CHA', 'CPA', 'CUI', 'DEB', 'FOC', 'GER', 'GOR', 'GUI', 'HOT', 'JEA', 'LUM', 'MAS', 'OGA', 'PER', 'REP', 'SAX', 'VAI', 'VMY']
Apps = ['Instagram', 'Deezer','Web_Weather','Web_Downloads','Google_Maps','Twitter','Tor','Microsoft_Office']

for app_j in Apps:
    for station_i in ['AMP','CHA','GER']:
        pd.DataFrame(data_dict_eps300[station_i][app_j],index = coverage_local).iloc[:96*7].plot(title = f"Station: {station_i}, app: {app_j}")

## Get Netmob from POIs:
Légèrement différent. Nécessite d'avoir accès en parallèle à un geopandas pour savoir de quel OSMID on parle:

In [ ]:
# POIs:
POI_types = ['iris','nightclub','park','parkings','public_transport','shop','stadium','station','theatre','transit','university']
Apps = ['Facebook','Google','Instagram','LinkedIn','Messenger','Snapchat','Twitter','WhatsApp']
POI_type = POI_types[0]
app_j = Apps[0]
transfer_mode = 'DL'

In [ ]:
# Aggregated data: 
path_agg_POI = f'POIs/netmob_POI_Lyon/Inputs/agg_TS/{POI_type}/{station_i}/{transfer_mode}'
data_path_agg,metadata_path_agg = f'{path_agg_POI}/data.npy', f'{path_agg_POI}/metadata.pkl' #Contains TS from all osmid of the same POI type

In [ ]:
# Time Series of POIs data: 
path_POI = f'POIs/netmob_POI_Lyon/Inputs/{station_i}'
data_path,metadata_path = f'{path_POI}/data.npy', f'{path_POI}/metadata.pkl' #contains 68 apps, all POIs

In [ ]:
## Time series data
f'POIs/netmob_POI_Lyon/{POI_type}/{app_j}/df_{osmid}_{transfer_mode}.csv'

## rien a voir

In [13]:
import pandas as pd
import os
import sys
import glob
path = '/home/rrochas/prediction-validation/save/K_fold_validation/training_with_HP_tuning/ASTGCN_2025_04_21_20_06_76371/'

list_csv = glob.glob(f"{path}*")
metrics_csv = [csv_path for csv_path in list_csv if 'METRICS_s' in csv_path]
losses_csv = [csv_path for csv_path in list_csv if 'Losses' in csv_path]

df_tot = pd.DataFrame()
for csv_path in metrics_csv:
    df_results = pd.read_csv(csv_path,index_col=0)
    trial_name = csv_path.split('76371')[-1][:-4]
    df_results['trial_name'] = trial_name
    df_tot = pd.concat([df_tot,df_results])
df_tot.loc['test'].set_index('trial_name')

,mse,mae,mape,mase,VAR_mse,VAR_mae,VAR_mape,VAR_mase,mse_complete_ds,mae_complete_ds,mape_complete_ds,mase_complete_ds
trial_name,,,,,,,,,,,,
subway_in_subway_out,1395.589233,22.009519,33.414791,0.714845,7109.694205,0.474380,11.462296,0.001928,1385.177246,21.834917,30.174503,0.681952
subway_in_subway_in,1437.444604,22.288612,32.871791,0.724384,5295.620212,0.395379,13.998599,0.002534,1435.644897,21.759230,30.017078,0.679505
station_epsilon100_Google_Maps,1531.804443,23.109518,34.713005,0.750498,10953.057676,0.982966,17.098634,0.002452,1477.172485,22.344402,29.677530,0.697943
station_epsilon100_Web_Weather,1417.820923,22.306835,34.520028,0.724395,229.888862,0.065928,9.670976,0.001280,1436.811401,22.356100,35.271950,0.698285
station_epsilon100_Web_Downloads,1455.519849,22.664582,35.607247,0.736529,3303.643634,0.471216,40.999183,0.002557,1460.185547,22.642090,34.627018,0.707224
station_epsilon100_Web_Deezer,1447.048975,22.571531,33.928539,0.733272,553.636681,0.117010,22.042596,0.001869,1465.362671,22.300707,31.896473,0.696368


In [18]:
import sys
import os
import pickle 
current_path = notebook_dir = os.getcwd()
working_dir = os.path.abspath(os.path.join(current_path, '..','..'))
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)

L_df_losses = []
for csv_path in losses_csv:
    df_losses = pd.read_csv(csv_path,index_col=0)
    df_losses = df_losses[['fcomplete_dataset_train_loss','fcomplete_dataset_valid_loss']]
    df_losses=df_losses.rename(columns={'fcomplete_dataset_train_loss':f'Train {trial_name}',
    'fcomplete_dataset_valid_loss':f'Valid {trial_name}'})
    L_df_losses.append(df_losses)


from plotting.TS_analysis import plot_TS

plot_TS(L_df_losses,width=1200,height=400,bool_show=True,title=f"Losses",x_datetime = False)

Loading BokehJS ...

figure(id='p1001', ...)